In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/DataSet_Titanic.csv")

In [3]:
import joblib
from sklearn.ensemble import RandomForestClassifier

random_forest_model: RandomForestClassifier = joblib.load(
    "D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

In [4]:
from sklearn.model_selection import train_test_split

data = df.drop(columns="Sobreviviente")
x_train, x_test, y_train, y_test = train_test_split(
    data, df["Sobreviviente"], test_size=0.2, random_state=123
)

In [5]:
q_variables_values_list = [
    {
        "column_name": "Genero",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Masculino"
            },
            {
                "old_value": 1,
                "new_value": "Femenino"
            },
        ]
    },
    {
        "column_name": "Clase",
        "variables": [
            {
                "old_value": 1,
                "new_value": "Primera Clase"
            },
            {
                "old_value": 2,
                "new_value": "Segunda Clase"
            },
            {
                "old_value": 3,
                "new_value": "Tercera Clase"
            },
        ]
    }
]

target_description = [
    {
        "column_name": "Sobreviviente",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Muere"
            },
            {
                "old_value": 1,
                "new_value": "Vive"
            },
        ]
    }
]

In [6]:
from sklearn.tree import _tree

DT = {
    # "model": tree.DecisionTreeClassifier(max_depth=2).fit(x_train, y_train),
    "model": random_forest_model.estimators_[25],
    "features": data.columns,
    "class_names": ["Muere", "Vive"],
    "target": "Sobreviviente",
    "q_variables": ["Genero", "Clase"],
    "q_variables_values": q_variables_values_list,
}

FUNCTION FOR GETTING RULES

In [30]:
def get_rules(model, q_variables, q_variables_values, features, class_names, target):
    tree_ = model
    feature_name = [
        features[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    cont = 1

    def get_simplified_route(route):
        simplified_route = list(route)
        watched_classes_plus = []
        watched_classes_less = []
        done = False
        iteration = 0
        while (not done):
            analyzed_rule = {}
            analyzed_item = ""
            analyzed_sign = ""
            analyzed_value = []
            analyzed_type = ""
            for i in range(len(route)):
                if (iteration + i) < len(route):
                    actual = route[i + iteration]
                    if actual["item"] not in q_variables and (analyzed_type == "number" or not analyzed_type):
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "number"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                            if analyzed_item not in watched_classes_plus and actual["sign"] == " > ":
                                watched_classes_plus.append(analyzed_item)
                                analyzed_sign = " > "
                            elif analyzed_item not in watched_classes_less and actual["sign"] == " <= ":
                                watched_classes_less.append(analyzed_item)
                                analyzed_sign = " <= "
                        elif actual["item"] == analyzed_item and actual["sign"] == analyzed_sign:
                            try:
                                if analyzed_sign == " > ":
                                    if actual["value"] > analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] <= analyzed_value:
                                        simplified_route.remove(actual)
                                elif analyzed_sign == " <= ":
                                    if actual["value"] <= analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] > analyzed_value:
                                        simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                    elif analyzed_type == "object" or not analyzed_type:
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "object"
                            analyzed_sign = "es :"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                        elif actual["item"] == analyzed_item:
                            try:
                                if all(val in analyzed_value for val in actual["value"]):
                                    simplified_route.remove(analyzed_rule)
                                    analyzed_value = actual["value"]
                                elif all(analyzed_val in actual["value"] for analyzed_val in analyzed_value):
                                    simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                else:
                    break
            iteration += 1
            if iteration >= len(route):
                done = True
        return simplified_route

    def get_rule(name, threshold, bigger):
        rule = [{
            "item": name,
            "sign": "",
            "value": []
        }]
        if name in q_variables:
            new_args = []
            first = True
            # condition = "es : "
            rule[0]["sign"] = "es: "
            for variables in q_variables_values:
                if variables["column_name"] == name:
                    for values in variables["variables"]:
                        if (not bigger and values["old_value"] <= threshold) or (
                                bigger and values["old_value"] > threshold):
                            new_args.append(values["new_value"])
            rule[0]["value"] = new_args
        else:
            if not bigger:
                rule[0]["sign"] = " <= "
            else:
                rule[0]["sign"] = " > "
            rule[0]["value"] = [f"{np.round(threshold, 3)}"]

        return rule

    def get_text_route(route):
        text_route = []
        for val in route:
            if val["item"] not in q_variables:
                text_route += [(f"({val['item']} {val['sign']} {val['value'][0]})")]
            else:
                values = ""
                first = True

                for i in range(len(val['value'])):
                    if first:
                        values += f"{val['value'][i]}"
                        first = False
                    else:
                        values += f" o {val['value'][i]}"
                text_route += [(f"{val['item']} {val['sign']} {values}")]
        return text_route

    def recurse(node, route, routes):

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path_rules, right_path_rules = list(route), list(route)
            left_path_rules += get_rule(name, threshold, bigger=False)
            recurse(tree_.children_left[node], left_path_rules, routes)
            right_path_rules += get_rule(name, threshold, bigger=True)
            recurse(tree_.children_right[node], right_path_rules, routes)
        else:
            text_route_original = get_text_route(route)
            text_route_original += [(tree_.value[node], tree_.n_node_samples[node])]
            simplified_route = get_simplified_route(route)
            text_route = get_text_route(simplified_route) 
            simplified_route += [(tree_.value[node], tree_.n_node_samples[node])]
            print(simplified_route)
            text_route += [(tree_.value[node], tree_.n_node_samples[node])]
            routes += [text_route]

    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    rules = []
    for path in paths:
        rule = "Si "
        
        print(path)

        for p in path[:-1]:
            if rule != "Si ":
                rule += " y "
            rule += str(p)
        rule += " entonces "
        if class_names is None:
            rule += "respuesta: " + str(np.round(path[-1][0][0][0], 3))
        else:
            classes = path[-1][0][0]
            position = np.argmax(classes)
            rule += f"{target}: {class_names[position]} (con probabilidad: " \
                    f"{np.round(100.0 * classes[position] / np.sum(classes), 2)}%)"
        rule += f" | basado en {path[-1][1]:,} muestras"
        rules += [f"Regla {cont}: {rule}"]
        cont += 1

    return rules

In [32]:
rules = get_rules(model=DT["model"].tree_, q_variables=DT["q_variables"], q_variables_values=DT["q_variables_values"], features=DT["features"], class_names=DT["class_names"], target=DT["target"])
# for r in rules:
#     print(r)
# print()

[{'item': 'PadHij', 'sign': ' <= ', 'value': ['0.5']}, {'item': 'Genero', 'sign': 'es: ', 'value': ['Masculino']}]
[{'item': 'PadHij', 'sign': ' <= ', 'value': ['0.5']}, {'item': 'Genero', 'sign': 'es: ', 'value': ['Femenino']}]
[{'item': 'PadHij', 'sign': ' > ', 'value': ['0.5']}, {'item': 'Genero', 'sign': 'es: ', 'value': ['Masculino']}]
[{'item': 'PadHij', 'sign': ' > ', 'value': ['0.5']}, {'item': 'Genero', 'sign': 'es: ', 'value': ['Femenino']}]
['(PadHij  <=  0.5)', 'Genero es:  Masculino', (array([[0.83161512, 0.16838488]]), 188)]
['(PadHij  <=  0.5)', 'Genero es:  Femenino', (array([[0.27118644, 0.72881356]]), 78)]
['(PadHij  >  0.5)', 'Genero es:  Masculino', (array([[0.66666667, 0.33333333]]), 47)]
['(PadHij  >  0.5)', 'Genero es:  Femenino', (array([[0.18055556, 0.81944444]]), 46)]
